In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0,1'
import soundfile as sf

from transformers import Qwen3OmniMoeForConditionalGeneration, Qwen3OmniMoeProcessor
from qwen_omni_utils import process_mm_info

MODEL_PATH = "/home/vladimir_albrekht/projects/2025_sep_22_qwen3omni/ms_swift_training/approach_2_transformers_based/models/5B_small_v"
# MODEL_PATH = "Qwen/Qwen3-Omni-30B-A3B-Thinking"

model = Qwen3OmniMoeForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    dtype="auto",
    device_map="auto",
    attn_implementation="flash_attention_2",
)

processor = Qwen3OmniMoeProcessor.from_pretrained(MODEL_PATH)

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "audio", "audio": "/home/vladimir_albrekht/projects/2025_sep_22_qwen3omni/models/Qwen3-Omni-30B-A3B-Instruct/unity_codes/audio_chunks/chunk_1.wav"},
            {"type": "text", "text": "Transcribe the English audio into text."}
        ],
    },
]

# Set whether to use audio in video
USE_AUDIO_IN_VIDEO = False

# Preparation for inference
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios, images, videos = process_mm_info(conversation, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = processor(text=text, 
                   audio=audios, 
                   images=images, 
                   videos=videos, 
                   return_tensors="pt", 
                   padding=True, 
                   use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = inputs.to(model.device).to(model.dtype)

# Inference: Generation of the output text and audio
text_ids, audio = model.generate(**inputs, 
                                 speaker="Chelsie", 
                                 thinker_return_dict_in_generate=True,
                                 use_audio_in_video=USE_AUDIO_IN_VIDEO, return_audio=False)

text = processor.batch_decode(text_ids.sequences[:, inputs["input_ids"].shape[1] :],
                              skip_special_tokens=True,
                              clean_up_tokenization_spaces=False)
print(text)
# if audio is not None:
#     sf.write(
#         "output.wav",
#         audio.reshape(-1).detach().cpu().numpy(),
#         samplerate=24000,
    # )
    

/home/vladimir_albrekht/projects/2025_sep_22_qwen3omni/ms_swift_training/approach_2_transformers_based/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section', 'mrope_interleaved', 'interleaved'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section', 'interleaved'}
You are attempting to use Flash Attention 2 without specifying a torch dtype. This might lead to unexpected behaviour
Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.87s/it]
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the s

['pta的支持izza的高度以内以内的高度以内的高度以内的高度的高度以内的高度以内的高度以内ptaSendMessagepta的支持以内pta的高度以内pta的高度以内pta以内 metabol以内 metabol以内 metabol以内以内以内 metabol以内 metabol以内 metabol以内 metabol以内 metabol以内 metabol以内 metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabol metabolstpoleanoleanoleanoleanoleanoleanstpoleanoleanoleanstpoleanstpoleanstpol

In [17]:

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "audio", "audio": "/home/vladimir_albrekht/projects/2025_sep_22_qwen3omni/models/Qwen3-Omni-30B-A3B-Instruct/unity_codes/audio_chunks/chunk_1.wav"},
            {"type": "text", "text": "Transcribe the English audio into text."}
        ],
    },
]

# Set whether to use audio in video
USE_AUDIO_IN_VIDEO = False

# Preparation for inference
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios, images, videos = process_mm_info(conversation, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = processor(text=text, 
                   audio=audios, 
                   images=images, 
                   videos=videos, 
                   return_tensors="pt", 
                   padding=True, 
                   use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = inputs.to(model.device).to(model.dtype)

# Inference: Generation of the output text and audio
text_ids, audio = model.generate(**inputs, 
                                 speaker="Chelsie", 
                                 thinker_return_dict_in_generate=True,
                                 use_audio_in_video=USE_AUDIO_IN_VIDEO, return_audio=False)

text = processor.batch_decode(text_ids.sequences[:, inputs["input_ids"].shape[1] :],
                              skip_special_tokens=True,
                              clean_up_tokenization_spaces=False)
print(text)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


RuntimeError: Expected all tensors to be on the same device, but got mat2 is on cuda:0, different from other tensors on cuda:1 (when checking argument in method wrapper_CUDA_mm)

In [ ]:

class TextSFTDataset(Dataset):
    def __init__(self, data_path, processor, max_seq_length):
        self.processor = processor
        self.max_seq_length = max_seq_length
        # IGNORE_INDEX is not needed here anymore, the processor handles it

        logger.info(f"Loading data from {data_path}...")
        jsonl_files = sorted(Path(data_path).glob("*.jsonl"))
        if not jsonl_files:
            raise ValueError(f"No .jsonl files found in {data_path}")
        dsets = [load_dataset("json", data_files=str(f), split="train") for f in jsonl_files]
        self.raw_ds = concatenate_datasets(dsets)
        
        logger.info(f"Data loading complete. Loaded {len(self.raw_ds)} samples.")

    def __len__(self):
        return len(self.raw_ds)

    def __getitem__(self, i):
        item = self.raw_ds[i]
        
        # Prepare the conversation in the chat template format
        conversation = [
            {"role": "system", "content": item["system"]},
            {"role": "user", "content": item["user"]},
            {"role": "assistant", "content": item["assistant"]},
        ]
        
        # Apply the chat template to convert conversation to a formatted string
        text = self.processor.tokenizer.apply_chat_template(
            conversation,
            tokenize=False,  # Don't tokenize yet, just format
            add_generation_prompt=False  # We already have the assistant response
        )
        
        # Now tokenize the formatted text
        inputs = self.processor.tokenizer(
            text,
            truncation=True,
            padding=False,  # Padding will be done by the collator
            max_length=self.max_seq_length,
            return_tensors=None  # Return python lists for the dataset
        )
        
        # For training, we need to set labels
        # Copy input_ids to labels for language modeling
        inputs["labels"] = inputs["input_ids"].copy()
        
        return inputs
    
dataset = TextSFTDataset(data_path, processor, max_seq_length)
sample = dataset[0]
print(f"Sample keys: {sample.keys()}")
print(f"Input shape: {len(sample['input_ids'])}")


In [ ]:
from transformers.models.qwen3_omni_moe import Qwen3OmniMoeThinkerTextModel
from transformers.models.qwen3_omni_moe.configuration_qwen3_omni_moe import Qwen3OmniMoeTextConfig 

# 2. Get the original config and create a new, smaller version
original_text_config = model.config.thinker_config.text_config

# You can now create a new config with your desired smaller parameters
# We keep hidden_size the same to ensure compatibility with the lm_head
small_text_config = Qwen3OmniMoeTextConfig(
    hidden_size=original_text_config.hidden_size,          # Keep this the same
    vocab_size=original_text_config.vocab_size,            # Keep this the same
    num_hidden_layers=4,                                   # Reduced from 48
    num_attention_heads=16,                                # Reduced from 32
    num_key_value_heads=2,                                 # Reduced from 4
    intermediate_size=1024,                                # Reduced from 768
    num_experts=8,                                         # Reduced from 128
    num_experts_per_tok=2,                                 # Reduced from 8
    moe_intermediate_size=256,                             # Reduced from 768
    # You can keep other parameters the same or modify them as needed
    **{k: v for k, v in original_text_config.to_dict().items() if k not in 
       ['hidden_size', 'vocab_size', 'num_hidden_layers', 'num_attention_heads', 
        'num_key_value_heads', 'intermediate_size', 'num_experts', 
        'num_experts_per_tok', 'moe_intermediate_size']}
)

print("--- Initializing new, smaller Thinker LLM part from scratch ---")

# 3. Initialize the smaller Thinker LLM part from scratch using the new config
# This creates a new model with randomly initialized weights.
small_thinker_llm = Qwen3OmniMoeThinkerTextModel(small_text_config)


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section', 'mrope_interleaved', 'interleaved'}


--- Initializing new, smaller Thinker LLM part from scratch ---


In [11]:
total_params = sum(p.numel() for p in small_thinker_llm.parameters() if p.requires_grad)
total_params / 1_000_000_000

0.399592448

In [13]:
model.thinker.model = small_thinker_llm

In [18]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/vladimir_albrekht/projects/2025_sep_22_qwen3omni/ms_swift_training/approach_2_transformers_based/src/inference


Qwen3OmniMoeTextConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "decoder_sparse_step": 1,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "max_position_embeddings": 65536,
  "mlp_only_layers": [],
  "model_type": "qwen3_omni_moe_text",
  "moe_intermediate_size": 256,
  "norm_topk_prob": true,
  "num_attention_heads": 16,
  "num_experts": 8,
  "num_experts_per_tok": 2,
  "num_hidden_layers": 4,
  "num_key_value_heads": 2,
  "output_router_logits": false,
  "rms_norm_eps": 1e-06,
  "rope_scaling": {
    "interleaved": true,
    "mrope_interleaved": true,
    "mrope_section": [
      24,
      20,
      20
    ],
    "rope_type": "default",
    "type": "default"
  },
  "rope_theta": 1000000,
  "router_aux_loss_coef": 0.001,
  "shared_expert_intermediate_size": 0,
  "sliding_window": null,
  "tf_legacy_loss": false,
  "tie_word_embeddings": false,
  "transformers_version": "4.57.0.dev0",
  "use_

In [21]:
model.config.thinker_config.text_config = small_thinker_llm.config

In [22]:
PATH = "/home/vladimir_albrekht/projects/2025_sep_22_qwen3omni/ms_swift_training/approach_2_transformers_based/models/5B_small_v"
model.save_pretrained(PATH)
processor.save_pretrained(PATH)

/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


[]

In [16]:
total_p = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_p /1_000_000_000

5.438191153

In [8]:
model.thinker.lm_head

Linear(in_features=2048, out_features=152064, bias=False)

In [17]:
model.config.thinker_config.text_config

Qwen3OmniMoeTextConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "decoder_sparse_step": 1,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 768,
  "max_position_embeddings": 65536,
  "mlp_only_layers": [],
  "model_type": "qwen3_omni_moe_text",
  "moe_intermediate_size": 768,
  "norm_topk_prob": true,
  "num_attention_heads": 32,
  "num_experts": 128,
  "num_experts_per_tok": 8,
  "num_hidden_layers": 48,
  "num_key_value_heads": 4,
  "output_router_logits": false,
  "rms_norm_eps": 1e-06,
  "rope_scaling": {
    "interleaved": true,
    "mrope_interleaved": true,
    "mrope_section": [
      24,
      20,
      20
    ],
    "rope_type": "default",
    "type": "default"
  },
  "rope_theta": 1000000,
  "router_aux_loss_coef": 0.001,
  "shared_expert_intermediate_size": 0,
  "sliding_window": null,
  "tf_legacy_loss": false,
  "tie_word_embeddings": false,
  "transformers_version": "4.57.0.dev0",
  "us

In [ ]:
model.

In [1]:
model.thinker.model

NameError: name 'model' is not defined

In [ ]:
total_params = sum(p.numel() for p in model.thinker.model.parameters() if p.requires_grad)
total_params / 1_000_000_000

# printed:
# >> 30B

30.22121984

In [10]:
total_params = sum(p.numel() for p in model.talker.parameters() if p.requires_grad)
total_params / 1_000_000_000

3.32459648

31.719205488